# Querying and Creating Portfolios in VAM Client
This notebook serves as a tutorial for querying, creating, and managing portfolios using the `mainsequence.client` package. It covers:
- Querying existing portfolios using filters
- Creating portfolios from time series
- Creating an index asset portfolio
- Querying the newly created index asset portfolio

## Step 1: Import Required Modules
First, we import the necessary modules and constants.

In [1]:
import os
from pathlib import Path
import dotenv
# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')
from mainsequence.client import DataUpdates
import datetime
from mainsequence.client import TargetPortfolio, Asset
from mainsequence import MARKETS_CONSTANTS 


Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


## Step 2: Query Portfolios
We query an existing portfolio using the `local_time_serie_hash_id` filter. This returns a single portfolio object.

In [2]:

# Query a single TargetPortfolio with a specific ticker
portfolio_ticker = "TICKER1"
portfolio = TargetPortfolio.filter(portfolio_ticker=portfolio_ticker)
print("Queried Portfolio:", portfolio)


Queried Portfolio: []


## Step 3: Create Portfolios from Time Series
We create two portfolios for different purposes:
- **Execution Portfolio** for real-time execution
- **Backtesting Portfolio** for simulation and historical analysis

### !!! Important !!!

If you want to start building portfolios we recommend you to use our VirtualFundBuilder. This will help you have a more fluid 
and seamless workflows. In this example we will show you how to build a portfolio without using our Wrapper. A few important considerations

1.  Each portfolio requires a `local_signal_time_serie` and a `local_time_serie` that should come from TDAG, this is a unique identifier in the backend and placing a random integer will likely collide therefore we need to build a time serie that has a time_index and at least a column with a column "close" 



In [3]:
from mainsequence.tdag.time_series import TimeSerie
from mainsequence.client import DataUpdates
import numpy as np
import datetime
import pandas as pd

class PortfolioExample(TimeSerie):
    
    @TimeSerie._post_init_routines()
    def __init__(self, portfolio_volatility: float,source_prices:str, *args, **kwargs):
        self.portfolio_volatility = portfolio_volatility
        self.source_prices=source_prices
        super().__init__(*args, **kwargs)
        
    def update(self, update_statistics):
        """
        Update the time series by simulating a new data point based on the last observation,
        or simulate a complete series if no observations exist.

        :param update_statistics: A dict to record update statistics (optional).
        :return: pd.DataFrame with index as a timezone-aware datetime (UTC) and a column "close".
        """
        last_observation = self.get_last_observation()
        dt = 1.0  # one day time increment

        if last_observation is not None:
            return pd.DataFrame() # do not make sequential updats for example
          
        else:
            # If no observation exists, simulate a daily series for the last 30 days.
            end_date = datetime.datetime.now(datetime.timezone.utc)
            start_date = end_date - datetime.timedelta(days=30)
            # Generate a date range with daily frequency (timezone-aware in UTC).
            dates = pd.date_range(start=start_date, end=end_date, freq='D', tz=datetime.timezone.utc)
            
            # Initialize with a default price.
            prices = [100.0]
            for _ in range(1, len(dates)):
                random_return = np.random.normal(loc=0, scale=self.portfolio_volatility * np.sqrt(dt))
                new_price = prices[-1] * np.exp(random_return - 0.5 * self.portfolio_volatility**2 * dt)
                prices.append(new_price)
                
            new_data = pd.DataFrame({"close": prices}, index=dates)
        if last_observation is not None:
            new_data=new_data[new_data.index>update_statistics._max_time_in_update_statistics]
         
        return new_data
    
    
portfolio_ts=PortfolioExample(portfolio_volatility=.1,source_prices="backtest")
portfolio_ts.run(debug_mode=True,force_update=True)


2025-04-10T19:27:11.533478Z [warning  ] Your TimeSeries is not in a python module this will likely bring exceptions when running in a pipeline api_time_series=False application_name=ms-sdk data_source_id=1 job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=39 (at time_series.py:1770 in get_time_serie_source_code())
2025-04-10T19:27:11.540197Z [warning  ] Your TimeSeries is not in a python module this will likely bring exceptions when running in a pipeline api_time_series=False application_name=ms-sdk data_source_id=1 job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=39 (at time_series.py:1770 in get_time_serie_source_code())
Overriding of current TracerProvider is not allowed
2025-04-10T19:27:16.744964Z [info     ] Updating Local Time Series for  PortfolioExample http://127.0.0.1:8000/local-time-series/details/?local_time_serie_id=127  for first time 

In [4]:
#we can get our simulated prices from TDAG Backend
print(f"Data in time series",portfolio_ts)
portfolio_ts.get_df_between_dates()

Data in time series PortfolioExample http://127.0.0.1:8000/local-time-series/details/?local_time_serie_id=127


,close
time_index,
2025-03-11 19:21:16.768000+00:00,100.000000
2025-03-11 19:24:06.269000+00:00,100.000000
2025-03-11 19:27:17.342000+00:00,100.000000
2025-03-12 19:27:17.342000+00:00,104.024793
2025-03-13 19:27:17.342000+00:00,113.061078
2025-03-14 19:27:17.342000+00:00,107.005609
2025-03-15 19:27:17.342000+00:00,99.468722
2025-03-16 19:27:17.342000+00:00,97.331693
2025-03-17 19:27:17.342000+00:00,104.889011


In [5]:

def create_portfolio(build_purpose, portfolio_name,portfolio_ts,):
    
    
    existing_portfolios = TargetPortfolio.filter(
       
        local_time_serie__id=portfolio_ts.local_metadata.id
    )
    
    if existing_portfolios:
        print(f"Portfolio '{portfolio_name}' already exists.")
        return existing_portfolios[0]
    
    
    return TargetPortfolio.create_from_time_series(
        portfolio_name=portfolio_name,
        build_purpose=build_purpose,
        is_active=True,
        local_time_serie_id=portfolio_ts.local_metadata.id,  # Example ID
        signal_local_time_serie_id=portfolio_ts.local_metadata.id, #we are not using a signal so we are setting the same 
        required_venues__symbols=[MARKETS_CONSTANTS.BINANCE_EV_SYMBOL],
        calendar_name="24/7",
        tracking_funds_expected_exposure_from_latest_holdings=False, 
        is_asset_only=False,
        backtest_table_price_column_name="close",
        target_portfolio_about=dict(description= "Test Portfolio For example",
                                    signal_name= "No signal",
                                    signal_description="no descritpiont",
                                    rebalance_strategy_name="no rebalance")
    )


# Create Backtesting Portfolio
backtest_portfolio = create_portfolio(
    MARKETS_CONSTANTS.PORTFOLIO_BUILD_FOR_BACKTEST, "Example Portfolio",portfolio_ts,
)
print("Created Backtest Portfolio:", backtest_portfolio)


ValidationError: 2 validation errors for TargetPortfolio
available_in_venues
  Field required [type=missing, input_value={'id': 32, 'portfolio_nam...e': 'close', 'tags': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
required_venues
  Extra inputs are not permitted [type=extra_forbidden, input_value=[9], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

126

In [ ]:
portfolio_ts

## Step 4: Create Index Asset Portfolio
An **Index Asset Portfolio** is created by linking both execution and backtesting portfolios.

In [7]:
valuation_asset=Asset.filter(symbol="USD",asset_type=MARKETS_CONSTANTS.ASSET_TYPE_CURRENCY,
                             execution_venue__symbol=MARKETS_CONSTANTS.MAIN_SEQUENCE_PORTFOLIOS_EV)
valuation_asset=valuation_asset[0]

## Step 5: Query the Newly Created Index Asset Portfolio
We verify the creation by querying the portfolio.

In [8]:


index_asset_portfolio = Asset.create_or_update_index_asset_from_portfolios(
    live_portfolio=execution_portfolio.id,
    backtest_portfolio=backtest_portfolio.id,
    valuation_asset=valuation_asset.id,
    calendar="24/7",
)

print("Created Index Asset Portfolio:", index_asset_portfolio)


Created Index Asset Portfolio: orm_class='TargetPortfolioIndexAsset' id=27043 symbol='LPQY7F-612-execution_reference/FBXUKW-611-backtesting_reference' name='Asset Tracking portfolios Execution Portfolio' asset_type='index' can_trade=False calendar=Calendar: 1 execution_venue=ExecutionVenue: 5 delisted_datetime=None unique_identifier='LPQY7F-612-execution_reference/FBXUKW-611-backtesting_reference_index_main_sequence_portfolios' unique_symbol='LPQY7F-612-execution_reference/FBXUKW-611-backtesting_reference' valuation_asset=AssetMixin: 15510 live_portfolio=TargetPortfolio: 682 backtest_portfolio=TargetPortfolio: 685


## Step 6: Adding Weights to the Portfolios
When using the VirtualFundBuilder signal and time series composition, there is no need to add weights. However,
if we want to add weights to our portfolios without using VirtualFundBuilder, we can do this manually.

Once the portfolio is created, we need to build the FrontEndDetails. This will allow us to link our assets and portfolios in the graphical user interface.


In [9]:
from mainsequence.vam_client.models import TargetPortfolioFrontEndDetails

def ensure_front_end_details(portfolio):
    front_end_detail_portfolio = TargetPortfolioFrontEndDetails.filter(target_portfolio__id=portfolio.id)
    
    if len(front_end_detail_portfolio) == 0:
        front_end_details_kwargs = {
            "target_portfolio_about": {
                "description": f"Test Example Portfolio {portfolio.portfolio_ticker}",
                "signal_name": "Manual Signal",
                "signal_description": """## Explanation
                Markdown Explanation example
                """,
                "rebalance_strategy_name": "Manual Rebalance",
            }
        }
        front_end_detail_portfolio = TargetPortfolioFrontEndDetails.create_or_update(
            **front_end_details_kwargs,
            target_portfolio_id=portfolio.id,
            backtest_table_time_index_name="time_index",
            backtest_table_price_column_name="portfolio",
            tags=["example portfolios"]
        )
    return front_end_detail_portfolio

# Apply function to both live and backtest portfolios
ensure_front_end_details(index_asset_portfolio.backtest_portfolio)
ensure_front_end_details(index_asset_portfolio.live_portfolio)

TargetPortfolioFrontEndDetails: 682

## Step 7: Adding Weights to the Portfolios
When using the VirtualFundBuilder signal and time series composition, there is no need to add weights. However,
if we want to add weights to our portfolios without using VirtualFundBuilder, we can do this manually.


In [10]:
TargetPortfolioFrontEndDetails.filter(target_portfolio__id=index_asset_portfolio.backtest_portfolio.id)

[TargetPortfolioFrontEndDetails: 685]

In [11]:
assets=Asset.filter(symbol__in=["BTCUSDT","ETHUSDT"],execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL)
assets=assets[:2]

In [12]:
np.random.rand()

0.7836151372395902

In [13]:
#Sends the calculated backtesting weights to the portfolio management system in VAM.
from mainsequence.vam_client import HistoricalWeights

#simulate weights mapping

for  weights_date in backtesting_portfolio_ts.get_df_between_dates().index:
    
    w=np.random.rand()
    
    positions_list=[{"weight_notional_exposure":(1-c)*w,"unique_identifier":a.unique_identifier} for c,a in enumerate(assets)]


    hw=HistoricalWeights.add_from_time_serie(
                        local_time_serie_id=backtesting_portfolio_ts.local_metadata.id,
                        weights_date=weights_date,
                        positions_list=positions_list
                    )

In [14]:
print("The latest weights of the portfolio can be found here",index_asset_portfolio.backtest_portfolio_details_url)

The latest weights of the portfolio can be found here http://192.168.178.69:8010/dashboards/portfolio-detail/?target_portfolio_id=685
